In [1]:
crimes <- read.csv("../data/crimes.csv")
crimes$region_central <- crimes$region == "central"
crimes$region_west <- crimes$region == "west"
# crimes$region_other <- crimes$region == "other"

crimes$region_central <- as.integer(as.logical(crimes$region_central))
crimes$region_west <- as.integer(as.logical(crimes$region_west))
# crimes$region_other <- as.integer(as.logical(crimes$region_other))

crimes <- subset(crimes, select = -c(region, name))
crimes$crimes <- round(log(crimes$crimes))
head(crimes)

,crimes,prbarr,prbpris,polpc,density,area,taxpc,pctmin,pctymale,wcon,wsta,wser,wtrd,wfir,region_central,region_west
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>
1,8,0.289696,0.472222,0.0017868,230.71590,423.943,25.69763,20.21870,0.0876968,206.4803,236.24,215.7335,182.3330,272.4492,1,0
2,6,0.202899,0.465753,0.0005939,97.68340,259.994,14.56088,7.91632,0.0870046,188.7683,247.38,191.3742,151.4234,202.4292,1,0
3,5,0.406593,0.500000,0.0008209,41.27659,235.059,18.63060,3.16053,0.0738525,147.9290,233.33,158.2278,143.5132,200.3205,0,1
4,6,0.431095,0.431373,0.0014327,48.21764,531.452,38.24473,47.91610,0.0836378,284.5809,206.07,208.4636,174.2457,207.1006,1,0
5,5,0.631579,0.383333,0.0008353,53.05164,426.135,16.85321,1.79619,0.0790580,195.9361,234.75,178.7785,152.7354,209.0301,0,1
6,6,0.369650,0.226415,0.0019067,59.10931,247.087,22.32144,1.54070,0.1063334,199.2377,227.69,176.5392,150.5835,195.3125,0,1


Vergleichen Sie die statistischen Eigenschaften (Einhaltung der angestrebten
Überdeckungswahrscheinlichkeit 1-$\alpha$) der in der Vorlesung vorgestellten
approximativen Konfidenzintervalle für die Koeffizienten mit denen der in R
verwendeten Profile-Likelihood-Konfidenzintervallen.

Konfidenzintervall für eine Komponente des Parametervektors
$$
P\left(\hat{\beta}_{j, n}-z_{1-\frac{a}{2}} \sqrt{I_{n}\left(\underline{\hat{\beta}}_{n}\right)_{j j}^{-1}}<\beta_{j}<\hat{\beta}_{j, n}+z_{1-\frac{\alpha}{2}} \sqrt{I_{n}\left(\underline{\hat{\beta}}_{n}\right)_{j j}^{-1}}\right) \approx 1-\alpha
$$

In [24]:
likelihood_confidence <- function(num_parameters, model, alpha){
    df <- as.data.frame(matrix(ncol = 2))
    colnames(df) <- c("2.5%", "97.5%")
    df <- df[0,]
    
    bound = 1 - alpha/2
    
    row_names <- c()
    
    for (i in 1:num_parameters){
        l_bound = coef(model)[i] - qnorm(bound) * sqrt(vcov(model)[i,i])
        u_bound = coef(model)[i] + qnorm(bound) * sqrt(vcov(model)[i,i])
        
        name = names(coef(model)[i])
        row_names <- append(row_names, name)
        
        df[nrow(df) + 1,] = c(l_bound,u_bound)
    }
    rownames(df) <- row_names
    return(df)
}

In [25]:
model = glm(crimes ~ 1+prbarr+prbpris+polpc+density+area+taxpc+region_central+region_west+pctmin+pctymale+wcon+wsta+wser+wtrd+wfir, data=crimes, family=poisson(link="log"))

# R Profile-Likelihood-Confidence Interval
r_conf <- confint(model)
# Own Likelihood-Confidence Intervall
my_conf <- likelihood_confidence(16, model, 0.05)

r_conf
my_conf
abs(r_conf - my_conf)

Waiting for profiling to be done...



,2.5 %,97.5 %
(Intercept),1.203520e-01,2.786869095
prbarr,-1.050336e+00,0.443780021
prbpris,-7.093033e-01,1.494875785
polpc,-7.845996e+01,48.152937028
density,-1.785680e-04,0.001423604
area,-2.317497e-04,0.000660978
taxpc,-1.328425e-02,0.011608856
region_central,-2.075617e-01,0.248016583
region_west,-3.547893e-01,0.295009215
pctmin,-7.270932e-03,0.007815839


,2.5%,97.5%
,<dbl>,<dbl>
(Intercept),1.163329e-01,2.782039760
prbarr,-1.042851e+00,0.450563571
prbpris,-7.131683e-01,1.490527170
polpc,-7.527264e+01,50.804510424
density,-1.657875e-04,0.001434788
area,-2.316497e-04,0.000660530
taxpc,-1.297932e-02,0.011889233
region_central,-2.078861e-01,0.247521614
region_west,-3.547079e-01,0.294769349


,2.5%,97.5%
,<dbl>,<dbl>
(Intercept),4.019031e-03,4.829335e-03
prbarr,7.484300e-03,6.783550e-03
prbpris,3.864904e-03,4.348615e-03
polpc,3.187324e+00,2.651573e+00
density,1.278046e-05,1.118392e-05
area,1.000234e-07,4.479421e-07
taxpc,3.049270e-04,2.803776e-04
region_central,3.243814e-04,4.949690e-04
region_west,8.144494e-05,2.398661e-04


In [5]:
h <- function(x) {
exp(x)
}

h1 <- function(x) {
exp(x)
}

In [72]:
model <- glm(crimes ~ 1+prbarr+prbpris, data=crimes, family=poisson(link="log"))
xneu <- c(1, 0.28, 0.48)
eta.hat <- predict(model, newdata=data.frame(prbarr=0.28, prbpris=0.48), type="response")
h(eta.hat) - qnorm(0.975) * sqrt(h1(eta.hat)^2 * t(xneu) %*% vcov(model) %*% xneu)
h(eta.hat) + qnorm(0.975) * sqrt(h1(eta.hat)^2 * t(xneu) %*% vcov(model) %*% xneu)

1451.105


1728.272


In [73]:
num_simulations <- 1000
set.seed(1)
result <- rep(0,num_simulations)
lower <- upper <- rep(0,num_simulations)
n <- nrow(crimes)
mu <- predict(model, type="response")

theo <-  predict(model, type="response", newdata=data.frame(prbarr=0.28, prbpris=0.48))
theo

1 
7.371293

In [74]:
for (i in 1:num_simulations)
    {
    ### Simulation von Pseudo-Beobachtungen
    y.sim <- rpois(n,mu)
    
    ### Schätzung der Parameter
    m <- glm(y.sim~1+prbarr+prbpris, data=crimes, family=poisson(link="log"))
    
    ### approx.Konfidenzintervall
    eta.hat <- predict(m, newdata=data.frame(prbarr=0.28, prbpris=0.48))
    lower[i] <- h(eta.hat) - qnorm(0.975)*sqrt(h1(eta.hat)^2*t(xneu) %*% vcov(m) %*% xneu)
    upper[i] <- h(eta.hat) + qnorm(0.975)*sqrt(h1(eta.hat)^2*t(xneu) %*% vcov(model) %*% xneu)
    
    ### theoretischer Wert enthalten?
    if ( (lower[i]<theo) && (theo<upper[i])) result[i] <- 1
}

In [75]:
mean(result)

[1] 0.952

In [21]:
cbind(lower, upper)[11:20,]

1 
7.371293

lower,upper
7.214441,8.557385
6.269153,7.496544
6.464344,7.713700
6.546190,7.807820
6.455568,7.702531
6.714566,7.993098
6.679225,7.959933
6.515891,7.776534
6.972986,8.288529
6.557126,7.819236
